<a href="https://colab.research.google.com/github/zia207/Survival_Analysis_Python/blob/main/Colab_Notebook/02_07_05_03_survival_analysis_absolute_risk_regression_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://drive.google.com/uc?export=view&id=1IFEWet-Aw4DhkkVe1xv_2YYqlvRe9m5_)

# 5.3 Absolute Risk Regression (Direct CIF Modeling)


Absolute Risk Regression** — also known as Direct CIF (Cumulative Incidence Function) Modeling— is a statistical approach used in competing risks survival analysis to model the absolute risk (probability)` of a specific event occurring by a certain time, directly.  Unlike traditional regression models that often focus on relative risks (how much more or less likely an event is in one group compared to another), absolute risk models aim to provide a direct estimate of the actual risk of an event.



## Overview


In competing risks survival data, an individual can experience `multiple types of events` — e.g., death from melanoma vs. death from other causes. The **Absolute Risk Regression (ARR)** model directly models the **Cumulative Incidence Function (CIF)** — that is, the absolute probability of a specific event by time `t`, given covariates. Unlike hazard models (Cox or Fine–Gray), ARR gives `direct probabilities`, which are easier to interpret in clinical and public health contexts. In competing risks, individuals may fail from one of several mutually exclusive causes (e.g., death from cancer vs. death from heart disease).


### What is Absolute Risk?


Imagine you're trying to predict the chance of someone developing a certain disease in the next 10 years.

`Relative Risk`: If Group A has a relative risk of 2 compared to Group B, it means people in Group A are twice as likely to develop the disease. However, if the baseline risk in Group B is very low (e.g., 1 in 10,000), then being "twice as likely" still means a very small absolute risk (2 in 10,000).

`Absolute Risk`: This is the actual probability. For example, "There is a 5% chance that this individual will develop the disease in the next 10 years." This is often more intuitive and directly relevant for patients and clinicians.



In contrast to the **Cox proportional hazards model**, which estimates *relative risk* (how much more likely one group is to experience an event compared to another), **absolute risk models estimate the actual risk (or hazard) on an absolute scale**, often expressed as:

- **Instantaneous event rate (hazard)** at time *t*, or  
- **Cumulative incidence**: the probability of experiencing the event by time *t*.

This makes them especially useful in **clinical decision-making**, **public health planning**, and **personalized risk prediction**, where knowing the *actual risk* (e.g., “You have a 15% chance of recurrence in 5 years”) is more actionable than a relative measure (e.g., “Your risk is 2× higher”).


In survival analysis with **competing risks**, absolute risk is quantified by the **Cumulative Incidence Function (CIF)** for event type \(k\):

$$
F_k(t) = P(T \leq t, \text{cause} = k)
$$

**Direct CIF modeling** regresses $F_k(t)$ directly on covariates $\mathbf{X}$, bypassing hazard modeling. A common approach uses **pseudo-values**:

1. Compute jackknife pseudo-values of the nonparametric CIF at fixed time points $t_1, \dots, t_K$:

$$
   \hat{\theta}_{i}(t) = n \hat{F}_k(t) - (n-1) \hat{F}_{k,-i}(t)
$$
   where $\hat{F}_{k,-i}(t)$ is the CIF leaving out subject $i$.

2. Regress pseudo-values on $\mathbf{X}$ using generalized estimating equations (GEE):

$$
   g\big( E[\hat{\theta}_{i}(t) \mid \mathbf{X}_i] \big) = \alpha(t) + \boldsymbol{\beta}(t)^\top \mathbf{X}_i
$$
   where $g(\cdot)$ is a link function (e.g., identity or logit).





### Advantages


- **Direct clinical interpretation**: “This treatment reduces absolute risk by 3% over 5 years.”
- **No proportional hazards assumption** (in time-varying versions).
- Better for **heterogeneous populations** where baseline risk varies widely.
- Naturally accommodates **time-varying effects**.


###  Challenges


- The hazard $\lambda(t \mid \mathbf{X})$ **must remain non-negative**—additive models can sometimes predict negative hazards if covariate effects are large (requires careful checking).
- Less familiar to many researchers compared to Cox models.
- Software implementation is less standardized (though `timereg`, `riskRegression`, and `cmprsk` in R support them).



## Cheeck and Load Required Packages

In [1]:
import subprocess
import sys

def install_and_import(package_name, import_name=None):
    """Install a package if not already installed and import it"""
    if import_name is None:
        import_name = package_name

    try:
        # Try to import the package
        globals()[import_name] = __import__(import_name)
        print(f"{package_name} is already installed")
    except ImportError:
        # If not installed, install it
        print(f"Installing {package_name}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            globals()[import_name] = __import__(import_name)
            print(f"{package_name} installed successfully")
        except Exception as e:
            print(f"Failed to install {package_name}: {e}")
            raise

# List of packages to check/install
packages = ['pandas', 'numpy', 'matplotlib', 'seaborn', 'scikit-learn', 'scikit-survival', 'lifelines', 'statsmodels']

# Install and import each package
for pkg in packages:
    try:
        install_and_import(pkg)
    except Exception as e:
        print(f"Error with package {pkg}: {e}")

# Verify installation by importing
print("Installed packages:")
for pkg in packages:
    try:
        __import__(pkg)
        print(f"✓ {pkg}")
    except ImportError:
        print(f"✗ {pkg}")

pandas is already installed
numpy is already installed
matplotlib is already installed
seaborn is already installed
Installing scikit-learn...
Failed to install scikit-learn: No module named 'scikit-learn'
Error with package scikit-learn: No module named 'scikit-learn'
Installing scikit-survival...
Failed to install scikit-survival: No module named 'scikit-survival'
Error with package scikit-survival: No module named 'scikit-survival'
Installing lifelines...
lifelines installed successfully
statsmodels is already installed
Installed packages:
✓ pandas
✓ numpy
✓ matplotlib
✓ seaborn
✗ scikit-learn
✗ scikit-survival
✓ lifelines
✓ statsmodels


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import proportional_hazard_test
import statsmodels.api as sm
from scipy import stats
from scipy.optimize import minimize
from scipy.special import expit  # logistic function
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

## Absolute Risk Regression in Python

Since, as of now, no mature Python package directly implements the `comp.risk()`-like functionality from R’s `timereg` for competing risks with additive and proportional absolute risk models, we will:

1. Load and prepare the data
2. Explain the four modeling approaches:
- Additive (logit link)
- Proportional absolute risk (identity link)
- Relative Cumulative Incidence Function (rcif)
- Logistic (logit with baseline CIF)
3. Implement core functions from scratch using maximum likelihood or constrained optimization
4. Fit models using scipy.optimize and demonstrate interpretation

### Data Preparation


n the period 1962-77, 205 patients with malignant melanoma (cancer of the skin) had a radical operation performed at Odense University Hospital, Denmark. All patients were followed until the end of 1977 by which time 134 were still alive while 71 had died (of out whom 57 had died from cancer and 14 from other causes).

`time`: time in days from operation

`status`: a numeric with values 0=censored 1=death.malignant.melanoma 2=death.other.causes

`event`: a factor with levels censored death.malignant.melanoma death.other.causes

`invasion`: a factor with levels level.0, level.1, level.2

`ici`: inflammatory cell infiltration (IFI): 0, 1, 2 or 3

`epicel`: a factor with levels not present present

`ulcer`: a factor with levels not present present

`thick`: tumour thickness (in 1/100 mm)

`sex`: a factor with levels Female Male

`age`: age at operation (years)

`logthick`: tumour thickness on log-scale




In [11]:
# Load data
url = "https://raw.githubusercontent.com/zia207/Survival_Analysis_Python/main/Data/Melanoma_data.csv"
melanoma = pd.read_csv(url)


# Define competing risks
# status: 0 = censored, 1 = melanoma death, 2 = other death
melanoma = melanoma.copy()
melanoma['cause'] = melanoma['status'].replace({1: 1, 2: 2, 0: 0})

# Covariates
categorical_covariates = ['sex', 'ulcer']
numerical_covariates = ['age', 'thick']

# One-hot encode categorical variables
X_categorical = pd.get_dummies(melanoma[categorical_covariates], drop_first=True)
X_numerical = melanoma[numerical_covariates]
X = pd.concat([X_numerical, X_categorical], axis=1).values

time = melanoma['time'].values
cause = melanoma['cause'].values

# Standardize numerical features

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

n = len(X)

## Understand Competing Risks Concept

In **absolute risk regression**, we model the **cumulative incidence function (CIF)** for cause ( k ) directly:

$$
F_k(t \| \mathbf{X}) = P(T \leq t, \text{cause} = k \| \mathbf{X})
$$



Unlike Cox models, this is **not conditional on survival**—it gives absolute risk.

Common link functions:

- **Identity**: $F_k(t\|\mathbf{X}) = F\_{k0}(t) + \mathbf{X}^\top \beta$ → **Proportional Absolute Risk** (must constrain ( 0 \leq F \leq 1 ))

- **Logit**: $\text{logit}(F_k(t\|\mathbf{X})) = \text{logit}(F\_{k0}(t)) + \mathbf{X}^\top \beta$ → **Additive (logit)**

- **Relative CIF**: $F_k(t\|\mathbf{X}) = F\_{k0}(t) \cdot \exp(\mathbf{X}^\top \beta)$ → but capped at 1 → less common

- **Logistic**: uses baseline CIF nonparametrically, then applies logit link with covariates

We’ll implement simplified **single-time-point** models for illustration (e.g., risk at ( t = 1825 ) days ≈ 5 years). Full time-varying CIF modeling requires splines or piecewise constants — beyond basic scope but extendable.

### Choose Time Horizon (e.g., 5 years = 1825 days)

In [12]:
t0 = 1825

# Assign outcome at t0
y = np.zeros(n, dtype=int)
for i in range(n):
    if time[i] <= t0:
        y[i] = cause[i]  # 1 or 2
    else:
        y[i] = 0  # censored at t0

# Event of interest: melanoma death
event = (y == 1).astype(int)

# Exclude subjects with competing event before t0 (simplified approach)
mask = (y != 2)
X_sub = X[mask]
event_sub = event[mask]
n_sub = len(X_sub)

print(f"Analyzing {n_sub} subjects (excluded {n - n_sub} with competing death before {t0} days)")

Analyzing 196 subjects (excluded 9 with competing death before 1825 days)


### Define Model Functions & Likelihood

$\text{logit}(F) = \text{logit}(F_0) + X^\top \beta \quad \Rightarrow \quad F = \text{expit}(\text{logit}(F_0) + X^\top \beta)$

Likelihood (binomial): $L = \prod_i F_i^{e_i} (1 - F_i)^{1 - e_i}$

But note: **subjects with competing events (y=2) should not be treated as "non-events"**!\
However, in **simplified direct binomial modeling**, we often **exclude competing events** or treat them as censored. For full proper CIF, use **Fine-Gray** or **direct binomial with cause-specific censoring**.

>  **Important**: For rigorous CIF modeling with competing risks, subjects with competing events **before t0** must be **excluded from the risk set for cause 1** *after* their event. But in a **single-time-point binomial approximation**, a common pragmatic approach is: - **Include only those not failed from competing cause before t0** - Or use **pseudo-values** (not covered here)



### Log-Likelihood for Logit (Additive) Model

In [13]:
def neg_loglik_logit(params, X, y):
    logit_F0 = params[0]
    beta = params[1:]
    linpred = logit_F0 + X @ beta
    F = expit(linpred)
    eps = 1e-10
    F = np.clip(F, eps, 1 - eps)
    ll = y * np.log(F) + (1 - y) * np.log(1 - F)
    return -np.sum(ll)

p = X_sub.shape[1]
init = np.zeros(p + 1)

res_logit = minimize(neg_loglik_logit, init, args=(X_sub, event_sub), method='L-BFGS-B')
beta_logit = res_logit.x[1:]
F0_logit = expit(res_logit.x[0])

print(f"Baseline 5-year risk (logit): {F0_logit:.4f}")
print("Coefficients (logit):", beta_logit)

Baseline 5-year risk (logit): 0.1812
Coefficients (logit): [0.12530994 0.54900454 0.18962116 0.76196331]


### Proportional Absolute Risk (Identity Link)

$F = F_0 + X^\top \beta, \quad \text{with } 0 \leq F \leq 1$

In [14]:
def neg_loglik_identity(params, X, y):
    F0 = params[0]
    beta = params[1:]
    F = F0 + X @ beta
    if np.any(F <= 0) or np.any(F >= 1):
        return 1e6
    eps = 1e-10
    F = np.clip(F, eps, 1 - eps)
    ll = y * np.log(F) + (1 - y) * np.log(1 - F)
    return -np.sum(ll)

init_id = np.concatenate([[event_sub.mean()], np.zeros(p)])
res_id = minimize(neg_loglik_identity, init_id, args=(X_sub, event_sub), method='L-BFGS-B')
beta_id = res_id.x[1:]
F0_id = res_id.x[0]

print(f"Baseline risk (identity): {F0_id:.4f}")
print("Coefficients (identity):", beta_id)

Baseline risk (identity): 0.2127
Coefficients (identity): [0.02224526 0.08172369 0.03073478 0.08859081]


### Nonparametric Baseline CIF Estimate

In [7]:
from lifelines import KaplanMeierFitter

# Crude CIF: treat competing events as censored
event_km = (melanoma['status'] == 1).astype(int)
time_km = melanoma['time'].copy()
time_km[melanoma['status'] == 2] = melanoma['time'][melanoma['status'] == 2]
event_km[melanoma['status'] == 2] = 0

kmf = KaplanMeierFitter()
kmf.fit(time_km, event_km)
F0_np = 1 - kmf.predict(t0)

print(f"Nonparametric crude CIF at {t0} days: {F0_np:.4f}")

Nonparametric crude CIF at 1825 days: 0.2313


### Logistic Model (Fixed Baseline CIF)

In [15]:
def neg_loglik_logistic_fixedF0(beta, X, y, F0):
    logit_F0 = np.log(F0 / (1 - F0))
    linpred = logit_F0 + X @ beta
    F = expit(linpred)
    eps = 1e-10
    F = np.clip(F, eps, 1 - eps)
    ll = y * np.log(F) + (1 - y) * np.log(1 - F)
    return -np.sum(ll)

init_beta = np.zeros(p)
res_logistic = minimize(neg_loglik_logistic_fixedF0, init_beta,
                        args=(X_sub, event_sub, F0_np), method='L-BFGS-B')
beta_logistic = res_logistic.x

print("Coefficients (logistic, fixed CIF):", beta_logistic)

Coefficients (logistic, fixed CIF): [0.11886064 0.52226175 0.16798616 0.65560896]


### Relative CIF (rcif)

Model: $F = F_0 \cdot \exp(X^\top \beta)$, truncated at 1.

In [16]:
def neg_loglik_rcif(beta, X, y, F0):
    F = F0 * np.exp(X @ beta)
    F = np.clip(F, 1e-10, 1 - 1e-10)
    ll = y * np.log(F) + (1 - y) * np.log(1 - F)
    return -np.sum(ll)

res_rcif = minimize(neg_loglik_rcif, np.zeros(p),
                    args=(X_sub, event_sub, F0_np), method='L-BFGS-B')
beta_rcif = res_rcif.x

print("Coefficients (rcif):", beta_rcif)

Coefficients (rcif): [0.0905864  0.24646605 0.11086705 0.42135408]


## Summary Comparison

In [18]:
models = {
    'Additive (Logit)': beta_logit,
    'Identity Link': beta_id,
    'Logistic (Fixed CIF)': beta_logistic,
    'Relative CIF (rcif)': beta_rcif
}

covariates = numerical_covariates + [col for col in X_categorical.columns] # Define covariates

print("\n=== Model Coefficients (standardized covariates) ===")
for name, beta in models.items():
    print(f"\n{name}:")
    for j, var in enumerate(covariates):
        print(f"  {var:6s}: {beta[j]: .4f}")


=== Model Coefficients (standardized covariates) ===

Additive (Logit):
  age   :  0.1253
  thick :  0.5490
  sex_Male:  0.1896
  ulcer_present:  0.7620

Identity Link:
  age   :  0.0222
  thick :  0.0817
  sex_Male:  0.0307
  ulcer_present:  0.0886

Logistic (Fixed CIF):
  age   :  0.1189
  thick :  0.5223
  sex_Male:  0.1680
  ulcer_present:  0.6556

Relative CIF (rcif):
  age   :  0.0906
  thick :  0.2465
  sex_Male:  0.1109
  ulcer_present:  0.4214


## Summary and Conclusion


We implemented four absolute risk regression approaches in Python for competing risks using the Melanoma dataset:

* Additive (logit): Flexible, ensures valid probabilities
* Identity: Direct risk difference, but requires constraints
* Logistic: Uses nonparametric baseline
* rcif: Relative scaling of baseline CIF

While no Python package currently replicates comp.risk() exactly, custom likelihood-based optimization with scipy provides a powerful and transparent alternative.

## Resources


* Gerds, T. A., & Andersen, P. K. (2012). *Absolute risk regression for competing risks: interpretation, link functions, and prediction.* **Statistics in Medicine**, 31(29), 3921–3930.
* `riskRegression` package documentation (CRAN): [https://cran.r-project.org/package=riskRegression](https://cran.r-project.org/package=riskRegression)

- Martinussen, T., & Scheike, T. H. (2006). *Dynamic Regression Models for Survival Data*. Springer.
- `timereg` package vignette: `vignette("timereg")`
- `riskRegression` package documentation